# Computing Objective Functions on the Raven server

Here we use birdy's WPS client to compare simulated and observed discharge using various objective functions on the server.

In [1]:
from birdy import WPSClient

from example_data import TESTDATA
import datetime as dt
from urllib.request import urlretrieve
import xarray as xr
import numpy as np
from matplotlib import pyplot as plt

url = "http://localhost:9099/wps"
wps = WPSClient(url)

# First, Run GR4JCN to generate a NetCDF file containing 'q_obs' and 'q_sim'.
This file contains 'q_obs' from the initial dataset, plus the simulated discharge 'q_sim' from the model.

In [2]:
# The model parameters. Can either be a string of comma separated values, a list, an array or a named tuple. 
params = '0.529, -3.396, 407.29, 1.072, 16.9, 0.947'

# Forcing files
ts=TESTDATA['raven-gr4j-cemaneige-nc-ts']

# Model configuration parameters
config = dict(
    start_date=dt.datetime(2000, 1, 1),
    end_date=dt.datetime(2002, 1, 1),
    area=4250.6,
    elevation=843.0,
    latitude=54.4848,
    longitude=-123.3659,
    )

# Let's call the model
resp = wps.raven_gr4j_cemaneige(ts=str(ts), params = params, **config)

# And get the response
# With `asobj` set to False, only the reference to the output is returned in the response. 
# Setting `asobj` to True will retrieve the actual files and copy the locally.
# Here we use false, as we do not want the file, we only want the path to the file.
[hydrograph, storage, solution, diagnostics] = resp.get(asobj=False)

Now we can call the objective function WPS service on the Raven Server and collect the results using 'asobj=True' to collect the actual results in a dict format.

In [3]:
resp = wps.objective_function(obs=hydrograph, sim=hydrograph)
results=resp.get(asobj=True)

Let's see the results!

In [4]:
print(results.metrics)

{'agreementindex': 0.4101473235671874, 'bias': 16.714572745371907, 'correlationcoefficient': 0.29808341759679685, 'covariance': 95.60942322512605, 'decomposed_mse': 1456.9251620291332, 'kge': -0.18901069937022497, 'log_p': -93.9590477440829, 'lognashsutcliffe': -1.768327662096807, 'mae': 20.36859544677684, 'mse': 1456.9251620291332, 'nashsutcliffe': -0.13031833498892342, 'pbias': -59.73471023716637, 'rmse': 38.16968904810639, 'rrmse': 1.364112232999017, 'rsquared': 0.08885372384618638, 'rsr': 1.0631643029132065, 'volume_error': -0.5973471023716637}


There are 17 objective functions. We can also call the WPS server with any one of those objective functions:

In [5]:
resp = wps.objective_function(obs=hydrograph, sim=hydrograph, name='agreementindex')
results=resp.get(asobj=True)
print(results.metrics)

{'agreementindex': 0.4101473235671874}
